In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from imblearn.over_sampling import SMOTE

In [4]:
df_train = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_train_PCA_dict_df_metab.csv')

In [5]:
y_train = df_train.iloc[:,0]
X_train = df_train.iloc[:,1:]

In [6]:
y_train = pd.get_dummies(y_train)

In [7]:
print (y_train.shape)
print (X_train.shape)
print (y_train.sum())

(128, 2)
(128, 88)
False    117
True      11
dtype: int64


In [49]:
smote = SMOTE(sampling_strategy=0.5)

In [50]:
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train[True])

In [51]:
print (X_train_smote.shape)
y_train_smote = y_train_smote.astype(bool)
not_y_train_smote = ~y_train_smote
y_train_smote_binary = pd.DataFrame({False:not_y_train_smote, True:y_train_smote})
print (y_train_smote_binary)

(175, 88)
     False  True 
0    False   True
1     True  False
2     True  False
3     True  False
4     True  False
..     ...    ...
170  False   True
171  False   True
172  False   True
173  False   True
174  False   True

[175 rows x 2 columns]


In [52]:
def build_model():
    model = keras.Sequential()
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=40, activation='tanh'))
    model.add(layers.Dense(units=20, activation='relu'))
    model.add(layers.Dense(units=20, activation='relu'))
    model.add(layers.Dense(2, activation = 'softmax')) #2 for binary classification
    
    model.compile(
        optimizer = keras.optimizers.RMSprop(
            learning_rate = 0.01,
            rho = 0.55,
            momentum = 0.5),
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [57]:
model = build_model()

In [58]:
class_weight = {False:1.0, True:3.0}
history = model.fit(X_train_smote.values, y_train_smote_binary.values, epochs=20)

Train on 175 samples
Epoch 1/20
175/175 [==============================] - 0s 868us/sample - loss: 0.6433 - acc: 0.6171
Epoch 2/20
175/175 [==============================] - 0s 102us/sample - loss: 0.2800 - acc: 0.8857
Epoch 3/20
175/175 [==============================] - 0s 106us/sample - loss: 0.1344 - acc: 0.9371
Epoch 4/20
175/175 [==============================] - 0s 109us/sample - loss: 0.0896 - acc: 0.9829
Epoch 5/20
175/175 [==============================] - 0s 101us/sample - loss: 0.0352 - acc: 0.9886
Epoch 6/20
175/175 [==============================] - 0s 109us/sample - loss: 0.0248 - acc: 0.9943
Epoch 7/20
175/175 [==============================] - 0s 104us/sample - loss: 0.0849 - acc: 0.9771
Epoch 8/20
175/175 [==============================] - 0s 105us/sample - loss: 0.0280 - acc: 0.9886
Epoch 9/20
175/175 [==============================] - 0s 107us/sample - loss: 0.0065 - acc: 1.0000
Epoch 10/20
175/175 [==============================] - 0s 103us/sample - loss: 0.0290 - 

In [43]:
df_dev = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_dev_PCA_dict_df_metab.csv')
print (df_dev.shape)

(37, 89)


In [44]:
y_dev = df_dev.iloc[:,0]
y_dev = pd.get_dummies(y_dev)
X_dev = df_dev.iloc[:,1:]

In [45]:
print (y_dev)
print (X_dev.shape)

    False  True 
0       1      0
1       1      0
2       1      0
3       0      1
4       0      1
5       1      0
6       0      1
7       1      0
8       1      0
9       1      0
10      1      0
11      1      0
12      1      0
13      1      0
14      1      0
15      1      0
16      1      0
17      1      0
18      1      0
19      1      0
20      1      0
21      1      0
22      1      0
23      1      0
24      1      0
25      1      0
26      1      0
27      1      0
28      1      0
29      1      0
30      1      0
31      1      0
32      1      0
33      1      0
34      1      0
35      1      0
36      1      0
(37, 88)


In [59]:
dev_pred = pd.Series(model.predict_classes(X_dev))
dev_y_vs_pred = pd.DataFrame({'actual':y_dev[True].values, 'pred':dev_pred})
print (dev_y_vs_pred)

    actual  pred
0        0     0
1        0     0
2        0     0
3        1     0
4        1     0
5        0     0
6        1     0
7        0     0
8        0     0
9        0     0
10       0     0
11       0     0
12       0     0
13       0     0
14       0     0
15       0     0
16       0     0
17       0     0
18       0     0
19       0     0
20       0     0
21       0     0
22       0     0
23       0     0
24       0     0
25       0     0
26       0     1
27       0     0
28       0     0
29       0     0
30       0     0
31       0     0
32       0     0
33       0     0
34       0     0
35       0     0
36       0     0
